# Predicting Video Game Sales

### Importing Modules

In [1]:
import re
import string
import pandas as pd
from sklearn import linear_model
import seaborn as sns
import numpy as np
import re
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.pipeline import Pipeline
import requests
from bs4 import BeautifulSoup
import io
from unidecode import unidecode
from sklearn.preprocessing import OneHotEncoder
import pickle
import warnings
warnings.filterwarnings('ignore')
import torch

### Exploratory Data Analysis (EDA)

In [2]:
# original dataset publicly available from kaggle.com

df = pd.read_csv('Data/vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


#### Which columns are features/targets/unnecessary for this project?
These conclusions were drawn from the dashboard webapp I developed, available [here](https://videogame-sales.streamlit.app/)

'Rank' - Unnecessary - It appears to rank the games on their global sales but an ML model will be able to do this on its own <br>
'Name' - Feature - Values containing popular characters (e.g. Mario) or belonging to popular serieses (e.g. Pokemon) tend to be better selling <br>
'Platform' - Feature - Certain platforms may have more/less users so games published for different platforms could affect sales <br>
'Year' - Feature - Could highlight trends over time in video game sales and can extrapolate trends for sales of future games <br>
'Genre' - Feature - Certain genres (e.g. Sports) could be more popular and therefore affect sales <br>
'Publisher' - Feature - Games from estabilished publishers with loyal followers are likely to have more sales <br>
'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'  - Targets - For this project, future sales in different regions need to be predicted using these columns
'Global_Sales' - Unnecessary - This column is a sum of other sales columns and can be dropped as it can be calculated after using a ML model to predict the other sales columns

#### What are limitations of the data?
The overall sales shown in the [dashboard](https://videogame-sales.streamlit.app/) seem to decrease after 2008 which can't be true as [video game sales have been known to be increasing consistently](https://www.washingtonpost.com/video-games/esports/2020/01/28/2010s-were-banner-decade-big-money-tech-esports-reaped-rewards/).
This may be attributed to the fact that [video game sales are becoming more and more digital](https://www.vintageisthenewold.com/game-pedia/what-percentage-of-game-sales-are-digital#:~:text=Do%20more%20people%20buy%20physical,were%20physical%20discs%20or%20cartridges.) through online platforms such as Steam, PSN, Xbox Live, and these platforms don't typically make sales information public which makes it difficult to keep track of sales. For this reason, the model trained on this data won't be useful in predicting overall sales data but can be useful in predicting physical sales but the user of the final webapp should be made aware of this.

In [3]:
# Drop unnecessary columns as deemed above

df = df.drop(columns=['Rank', 'Global_Sales'])

In [4]:
# Check if dataset contains significant size sample to train an ML model base on rule of thumb of 10x features

size = df.shape[0]
print('Dataset has '+ str(size) +' rows')
no_of_features = 5

if size > 10*no_of_features:
    print('Significant size sample')
else:
    print('Not enough data')

Dataset has 16598 rows
Significant size sample


In [5]:
# Check for missing values in all columns

missing_values_count = df.isnull().sum()
missing_values_count

Name             0
Platform         0
Year           271
Genre            0
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [6]:
# Find percent of data that is missing

total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells) * 100

print('Percent of data missing: ',str(percent_missing), '%')

Percent of data missing:  0.22024072512083115 %


Although percent of data missing is insignificant and could be ignored, these values may be able to obtained through web scraping

### Web Scraper to Impute Missing Values

'Year' and 'Publisher' column values can be scraped from metacritic

In [7]:
# Check if web scraper can connect to metacritic site

game = 'resident-evil-village' # random game for initial check
mc_url = 'https://www.metacritic.com/game/{}/'.format(game) # metacritic website url
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36)'}

response = requests.get(mc_url, headers=headers)

if response.status_code == 200:
    print('Successfully connected to Metacritic')
else:
    print('Could not connect')

Successfully connected to Metacritic


In [8]:
# Used in the next block of code to remove puncutation from the names of games so that the metacritic url can be generated as the url doesn't contain punctuation

remove = string.punctuation
remove = remove.replace("-", "").replace("&","") 
pattern = r"[{}]".format(remove)

In [9]:
# Initialises list which will keep track of games which still have missing info after scraping
missing_info_list = []

# Loop through each row in the dataset and generate the appropriate metacritic url and extract 'Year' and 'Publisher' values if missing 
for index, row in df.iterrows():
    try:
        if pd.isna(row.Year) or pd.isna(row.Publisher) or row.Publisher == 'Unknown':

            # replace punctuation in the name of the game so that it follows conventions of metacritic url
            game = re.sub(pattern, "", str(row.Name)).replace(" ", "-").replace("---", "-").replace("&","and").lower()
        
            mc_url = 'https://www.metacritic.com/game/{}/'.format(game)
            
            response = requests.get(mc_url, headers=headers)
            
            soup = BeautifulSoup(response.text, 'html.parser')
        
        
        if pd.isna(row.Year):

            # scrape year
            df.at[index,'Year']=int((soup.find('div', {'class': "c-gameDetails_ReleaseDate u-flexbox u-flexbox-row"})).text[-4:])

        if pd.isna(row.Publisher) or row.Publisher == 'Unknown':

            # scrap publisher
            df.at[index,'Publisher']=(soup.find('div', {'class': "c-gameDetails_Distributor u-flexbox u-flexbox-row"})).text[11:]

    except AttributeError as err1:

        # if unable to scrape any missing info, add name of game to missing_info list
        missing_info_list.append(str(row.Name))

In [10]:
# Percent of data that is still missing after web scraping

total_cells = np.product(df.shape)
total_missing = len(missing_info_list)
percent_missing = (total_missing/total_cells) * 100
print('Percent of data missing: ',str(percent_missing), '%')

Percent of data missing:  0.05823994858818332 %


Very insignificant percentage of data missing after scraping so it can be dropped

In [11]:
# Remove rows still with missing values and save filled-in data

df=df.dropna()
df.to_csv('Data/scraped_vgsales.csv', index=False)

### Preprocessing Data

In [12]:
# importing dataset from previous web-scraping section

df = pd.read_csv('Data/scraped_vgsales.csv')
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00


In [13]:
# standardise all column headers and 'string' dtype columns to be lowercase and unidecoded to ensure they are consistent

df.columns = map(str.lower, df.columns)
str_cols = ['name','platform','genre','publisher']
df[str_cols]=df[str_cols].astype(str).apply(lambda col: col.str.lower())
for col in str_cols:
    df[col] = df[col].apply(unidecode)
df.head()

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales
0,wii sports,wii,2006.0,sports,nintendo,41.49,29.02,3.77,8.46
1,super mario bros.,nes,1985.0,platform,nintendo,29.08,3.58,6.81,0.77
2,mario kart wii,wii,2008.0,racing,nintendo,15.85,12.88,3.79,3.31
3,wii sports resort,wii,2009.0,sports,nintendo,15.75,11.01,3.28,2.96
4,pokemon red/pokemon blue,gb,1996.0,role-playing,nintendo,11.27,8.89,10.22,1.00


#### Categorical Variable Encoding

To use the categorical variables (name, platform, genre, publisher) to train a ML model, they need to either be ordinally-encoded or one-hot encoded. None of these variables can be ordinally encoded as none of them have an inherent ranking (such as low, medium, high) so they have to be one-hot encoded and so the platform, genre and publisher columns will be one-hot encoded.

However, one-hot encoding would be unsuitable for the name column as each name is likely to be unique and this information won't help with predicting future games with different names. Instead, it's important to find out if the name belongs to a famous franchies (i.e. Mario or Pokemon) as this is likely to affect sales and will help predict sales of future games if they belong to any franchises.

In [14]:
# Wikipedia has a page with a list of best-selling video game franchises, this can be scraped to obtain a list which can be check against the names in the dataset to see if they belong to any of these

wiki_url="https://en.wikipedia.org/wiki/List_of_best-selling_video_game_franchises"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wiki_url)

# Check if scraper can connect to Wikipedia
if response.status_code == 200:
    print('Successfully connected to Wikipedia')
else:
    print('Could not connect')

Successfully connected to Wikipedia


In [15]:
soup = BeautifulSoup(response.text, 'html.parser')

# Create a dataframe which contains info extracted from tables on Wikipedia page
series_df =pd.DataFrame(columns=['Franchise name', 'Original release date', 'Sales'])
for i in range(4):
    table=soup.find_all('table',{'class':"wikitable"})[i]
    wiki_df=pd.read_html(io.StringIO(str(table)))
    wiki_df=pd.DataFrame(wiki_df[0])
    series_df=pd.concat([series_df, wiki_df])
series_df = series_df.drop(columns=['Unnamed: 3', 'Unnamed: 4','Unnamed: 5']).dropna()
series_df = series_df[series_df['Franchise name'] != series_df['Original release date']]

# Extract list of franchises from generated dataframe
franchise_list = [x.lower() for x in series_df['Franchise name']. tolist()]
for i, s in enumerate(franchise_list):
    franchise_list[i] = unidecode(s)

In [16]:
# Loop through each franchise in franchise_list and create a new column based on that franchise which gives binary True/False for each row depending on if the game in that row belongs to that franchise

pattern = '|'.join(franchise_list)
for item in franchise_list:
    df['series_'+item] = df['name'].str.contains(item).astype(int)
df = df.drop(columns=['name'])

df.head()

,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,series_mario,series_tetris,...,series_frogger,series_j.b. harold,series_lemmings,series_simple,series_singstar,series_spongebob squarepants,series_spyro,series_mass effect,series_simcity,series_watch dogs
0,wii,2006.0,sports,nintendo,41.49,29.02,3.77,8.46,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nes,1985.0,platform,nintendo,29.08,3.58,6.81,0.77,1,0,...,0,0,0,0,0,0,0,0,0,0
2,wii,2008.0,racing,nintendo,15.85,12.88,3.79,3.31,1,0,...,0,0,0,0,0,0,0,0,0,0
3,wii,2009.0,sports,nintendo,15.75,11.01,3.28,2.96,0,0,...,0,0,0,0,0,0,0,0,0,0
4,gb,1996.0,role-playing,nintendo,11.27,8.89,10.22,1.00,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Use scikit-learn's inbuilt one-hot encoder to encode the remaining categorical variables (platform, genre and publisher)

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
object_cols = ['platform','genre','publisher']
OH_cols = pd.DataFrame(OH_encoder.fit_transform(df[object_cols]))
OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)

OH_cols.index = df.index
df = df.drop(object_cols, axis=1)
df = pd.concat([df, OH_cols], axis=1)
df.columns = df.columns.astype(str)

df.head()

,year,na_sales,eu_sales,jp_sales,other_sales,series_mario,series_tetris,series_pokemon,series_call of duty,series_grand theft auto,...,publisher_yacht club games,publisher_yamasa entertainment,publisher_yeti,publisher_yoostar entertainment group,publisher_yuke's,publisher_yumedia,publisher_zenrin,publisher_zoo digital publishing,publisher_zoo games,publisher_zushi games
0,2006.0,41.49,29.02,3.77,8.46,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1985.0,29.08,3.58,6.81,0.77,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2008.0,15.85,12.88,3.79,3.31,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009.0,15.75,11.01,3.28,2.96,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1996.0,11.27,8.89,10.22,1.00,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Use scikit-learn's inbuilt minmaxscaler to scale the year column to the range of 0 to 1 to ensure it's considered equally to the other variables

year_scaler = MinMaxScaler()
na_scaler = MinMaxScaler()
eu_scaler = MinMaxScaler()
jp_scaler = MinMaxScaler()
other_scaler = MinMaxScaler()

df['year']=year_scaler.fit_transform(df[['year']])
df['na_sales']=na_scaler.fit_transform(df[['na_sales']])
df['eu_sales']=eu_scaler.fit_transform(df[['eu_sales']])
df['jp_sales']=jp_scaler.fit_transform(df[['jp_sales']])
df['other_sales']=other_scaler.fit_transform(df[['other_sales']])

df.head()

,year,na_sales,eu_sales,jp_sales,other_sales,series_mario,series_tetris,series_pokemon,series_call of duty,series_grand theft auto,...,publisher_yacht club games,publisher_yamasa entertainment,publisher_yeti,publisher_yoostar entertainment group,publisher_yuke's,publisher_yumedia,publisher_zenrin,publisher_zoo digital publishing,publisher_zoo games,publisher_zushi games
0,0.604651,1.000000,1.000000,0.368885,0.800378,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.116279,0.700892,0.123363,0.666341,0.072848,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.651163,0.382020,0.443832,0.370841,0.313150,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.674419,0.379610,0.379394,0.320939,0.280038,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.372093,0.271632,0.306340,1.000000,0.094607,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Training & Validating Neural Network Model

In [19]:
# Split dataset into features and targets as decided previously
targets = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
X = df.drop(columns=targets)
y = df[targets]

# Split datasets into training and validation 
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [20]:
# Check to see if a GPU with cuda cores is available to use for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was detected.'.format(device)) 

# Print the name of the cuda device, if detected
if device=='cuda':
  print (torch.cuda.get_device_name(device=device))

A cuda device was detected.
NVIDIA GeForce RTX 4080


In [21]:
# Create PyTorch tensors and move to device found above

train_X = torch.tensor(train_X.values,dtype=torch.float, device=device)
train_y = torch.tensor(train_y.values,dtype=torch.float, device=device)

In [22]:
# Defining PyTorch neural network model and moving to device
model = torch.nn.Sequential(
          torch.nn.Linear(X.shape[1], 1500),
          torch.nn.ReLU(),
          torch.nn.Linear(1500, y.shape[1])
        )
model.to(device)

Sequential(
  (0): Linear(in_features=782, out_features=1500, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1500, out_features=4, bias=True)
)

In [23]:
# Crtiterion for neural network to keep track of performance
criterion = torch.nn.MSELoss()

# SGD optimisation 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training network using entire dataset 50 times
for epoch in range(50):
    total_loss = 0
    for i in range(len(train_X)):
        # Single Forward Pass
        pred_y = model(train_X[i])

        # Measure how well the model predicted vs the actual value
        loss = criterion(pred_y, train_y[i])
        total_loss+=loss.item()

        # Update  neural network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss after each training iteration
    print ("Total Loss: ", total_loss)

Total Loss:  5.816181156674489
Total Loss:  4.958172704203548
Total Loss:  4.632962087982474
Total Loss:  4.411337113388868
Total Loss:  4.233746146693981
Total Loss:  4.08311886216412
Total Loss:  3.9498854446298566
Total Loss:  3.829674914460554
Total Loss:  3.7221423861454497
Total Loss:  3.62391338090627
Total Loss:  3.533942801161885
Total Loss:  3.452316255561197
Total Loss:  3.378064530429427
Total Loss:  3.3090722132394745
Total Loss:  3.2466006996656205
Total Loss:  3.1878946345924106
Total Loss:  3.1340128823882765
Total Loss:  3.0845177269279844
Total Loss:  3.038385011441953
Total Loss:  2.997027549646784
Total Loss:  2.957210371013886
Total Loss:  2.9207361653778534
Total Loss:  2.8865412157873784
Total Loss:  2.8552161808111336
Total Loss:  2.8247573358605393
Total Loss:  2.797776264427398
Total Loss:  2.771752012274657
Total Loss:  2.74707105863547
Total Loss:  2.7259885787654525
Total Loss:  2.7050057210654046
Total Loss:  2.683402152049588
Total Loss:  2.66507187363166

In [24]:
# Create input tensor
val_X = torch.tensor(val_X.values,dtype=torch.float, device=device)

# Pass the input into the neural network
pred_y = model(val_X)

# Measuring final model's MAE by comparing values predicted by model and actual values
mean_absolute_error(val_y, pred_y.cpu().detach().numpy())

0.007181461063663506

### Outputting Variables for Webapp

In [25]:
pickle.dump(model, open('Pickle/model.sav', 'wb'))

pickle.dump(year_scaler, open('Pickle/year_scaler.sav', 'wb'))
pickle.dump(na_scaler, open('Pickle/na_scaler.sav', 'wb'))
pickle.dump(eu_scaler, open('Pickle/eu_scaler.sav', 'wb'))
pickle.dump(jp_scaler, open('Pickle/jp_scaler.sav', 'wb'))
pickle.dump(other_scaler, open('Pickle/other_scaler.sav', 'wb'))



predictor_df = X[0:1]*0
predictor_df.to_pickle('Pickle/predictor_df.sav')

with open('Pickle/franchise_list.pkl', 'wb') as f:
    pickle.dump(franchise_list, f)